# Objective
In this notebook I will be creating an MLP that transforms a <a href="https://nlp.stanford.edu/projects/glove/">GloVe embedding</a> into a dense image embedding.  This will ultimately be a critical piece in our effort to generate images from arbitrary text.

# Data

We will be using the Common Object in Context <a href="http://cocodataset.org/#home">(coco)</a> dataset to train this network:

###### download these into a data/ folder and rename the folders to data/images/ and data/captions respectively
<a href="http://images.cocodataset.org/zips/train2014.zip">
    data/images
</a>
<br>
<a href="http://images.cocodataset.org/annotations/annotations_trainval2014.zip">
    data/captions
</a>
###### data/ is already in my .gitignore

# Architecture
Here is a visual depiction of the architecture of the network we will be creating:
<img src="img/architecture.png">

# Training
To train this network, we will be treating the image embedding as our prediction.  Our target will be the values of one of the layers deep in the VGG 18 network for when we run the captioned image.

# Loading daters

In [8]:
import json as jason
annotations = jason.loads("data/annotations/captions_train2014.json")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Loss

In [2]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, Input
from keras.utils.data_utils import get_file
import keras.backend as K
import h5py
import numpy as np
import tensorflow as tf

WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'

MEAN_PIXEL = np.array([103.939, 116.779, 123.68])

weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                        WEIGHTS_PATH_NO_TOP,
                        cache_subdir='models',
                        file_hash='253f8cb515780f3b799900260a226db6')
WEIGHTS_FILE = h5py.File(weights_path)

def vgg_layers(img_input, input_shape):
    # Block 1
    img_input = Input(tensor=img_input, shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    return x


def load_weights(model):
    f = WEIGHTS_FILE
    layer_names = [name for name in f.attrs['layer_names']]

    for layer in model.layers:
        b_name = layer.name.encode()
        if b_name in layer_names:
            g = f[b_name]
            weights = [g[name] for name in g.attrs['weight_names']]
            layer.set_weights(weights)
            layer.trainable = False


def VGG19(img_input, input_shape):
    """
    VGG19, but can take input_tensor, and load weights on VGG layers only
    """
    model = Model(img_input, vgg_layers(img_input, input_shape), name='vgg19')
    load_weights(model)
    return model


def preprocess_input(x):
    # Convert 'RGB' -> 'BGR'
    if type(x) is np.ndarray:
        x = x[..., ::-1]
    else:
        x = tf.reverse(x, [-1])

    return x - MEAN_PIXEL


In [4]:
from functools import reduce

from keras import backend as K
import numpy as np
import tensorflow as tf

def get_vgg_features(input, layers, input_shape):
    if len(K.int_shape(input)) == 3:
        input = K.expand_dims(input, axis=0)
    input = preprocess_input(input)
    vgg = VGG19(input, input_shape)
    outputs = [layer.output for layer in vgg.layers if layer.name in layers]
    return outputs
